[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/basf/mlipx/blob/main/docs/source/notebooks/structure_relaxation.ipynb)

# Structure Relaxtion with Custom Nodes

You can combine `mlipx` with custom code by writing ZnTrack nodes.
We will write a Node to perform a geometry relaxation similar to `mlipx.StructureOptimization`.

In [ ]:
# Only install the packages if they are not already installed
!pip show mlipx > /dev/null 2>&1 || pip install mlipx
!pip show rdkit2ase > /dev/null 2>&1 || pip install rdkit2ase

In [ ]:
# We will create a GIT and DVC repository in a temporary directory
import os
import tempfile

temp_dir = tempfile.TemporaryDirectory()
os.chdir(temp_dir.name)

Like all `mlipx` Nodes we will use a GIT and DVC repository to run experiments.
To make our custom code available, we structure our project like

```
relaxation/
   ├── .git/
   ├── .dvc/
   ├── src/__init__.py
   ├── src/relaxation.py
   ├── models.py
   └── main.py
```

to allow us to import our code `from src.relaxation import Relax`.
Alternatively, you can package your code and import it like any other Python package.

In [ ]:
!git init
!dvc init --quiet
!mkdir src
!touch src/__init__.py

The code we want to put into our `Relax` `Node` is the following:


```python
from ase.optimize import BFGS
import ase.io

data: list[ase.Atoms]
calc: ase.calculator.Calculator

end_structures = []
for atoms in data:
    atoms.set_calculator(calc)
    opt = BFGS(atoms)
    opt.run(fmax=0.05)
    end_structures.append(atoms)

ase.io.write('end_structures.xyz', end_structures)
```

To do so, we need to identify and define the inputs and outputs of our code.
We provide the `data: list[ase.Atoms]` from a data loading Node.
Therefore, we use `data: list = zntrack.deps()`.
If you want to read directly from file you could use `data_path: str = zntrack.deps_path()`.
We access the calculator in a similar way using `model: NodeWithCalculator = zntrack.deps()`.
`mlipx` provides the `NodeWithCalculator` abstract base class for a common communication on how to share `ASE` calculators.
Another convention is providing inputs as `data: list[ase.Atoms]` and outputs as `frames: list[ase.Atoms]`.
As we save our data to a file, we define `frames_path: pathlib.Path = zntrack.outs_path(zntrack.nwd / 'frames.xyz')` to store the output trajetory in the node working directory (nwd) as `frames.xyz`.
The `zntrack.nwd` provides a unique directory per `Node` to store the data at.
As the communication between `mlipx` nodes is based on `ase.Atoms` we define a `@frames` property.
Within this, we could also alter the `ase.Atoms` object, thus making the node communication independent of the file format facilitating data communication via code or Data as Code (DaC).
To summarize, each Node provides all the information on how to `save` and `load` the produced data, simplifying communication and reducing issues with different file format conventions.

Besides the implemented fields, there are also `x: dict = zntrack.params`, `x: dict = zntrack.metrics` and `x: pd.DataFrame = zntrack.plots` and their corresponding file path versions `x: str|pathlib.Path = zntrack.params_path`, `zntrack.metrics_path` and `zntrack.plots_path`.
For general outputs there is `x: any = zntrack.outs`. More information can be found at https://dvc.org/doc/start/data-pipelines/metrics-parameters-plots .

In [ ]:
%%writefile src/relaxation.py
import zntrack
from mlipx.abc import NodeWithCalculator
from ase.optimize import BFGS
import ase.io
import pathlib



class Relax(zntrack.Node):
    data: list = zntrack.deps()
    model: NodeWithCalculator = zntrack.deps()
    frames_path: pathlib.Path = zntrack.outs_path(zntrack.nwd / 'frames.xyz')

    def run(self):
        end_structures = []
        for atoms in self.data:
            atoms.set_calculator(self.model.get_calculator())
            opt = BFGS(atoms)
            opt.run(fmax=0.05)
            end_structures.append(atoms)
        with open(self.frames_path, 'w') as f:
            ase.io.write(f, end_structures, format='extxyz')
    
    @property
    def frames(self) -> list[ase.Atoms]:
        with self.state.fs.open(self.frames_path, "r") as f:
            return ase.io.read(f, format='extxyz', index=':')


With this Node definition, we can import the Node and connect it with `mlipx` to form a graph.

In [ ]:
from src.relaxation import Relax

import mlipx

In [ ]:
project = mlipx.Project()

emt = mlipx.GenericASECalculator(
    module="ase.calculators.emt",
    class_name="EMT",
)

with project:
    confs = mlipx.Smiles2Conformers(smiles="CCCC", num_confs=5)
    relax = Relax(data=confs.frames, model=emt)

project.build()

To execute the graph, we make use of `dvc repro` via `project.repro`.

In [ ]:
project.repro(build=False)

Once the graph has been executed, we can look at the resulting structures.

In [ ]:
relax.frames

In [ ]:
temp_dir.cleanup()